# Distributed Processing with Dask

This notebook demonstrates how to use Dask for distributed processing of large oceanographic datasets on the Ocean Data Platform.

**What you'll learn:**
- Set up a Dask client in ODP Workspaces
- Process larger-than-memory datasets using Dask DataFrames
- Parallelize computations across dataset partitions
- Scale aggregations and transformations for large data volumes

**Why Dask for Ocean Data?**

Research vessels can generate 25-30 TB of data per mission from sensors, ROVs, and sampling systems. Dask enables:
- Processing datasets larger than available RAM
- Parallel execution across multiple cores/workers
- Lazy evaluation - build computation graphs before executing
- Familiar pandas-like API

**Prerequisites:**
- Running in ODP Workspace (Dask sidecar pre-configured)
- Completed `01_catalog_discovery.ipynb` to understand dataset access

## 1. Setup and Dask Client Initialization

In [ ]:
import dask
import dask.dataframe as dd
from dask.distributed import Client, progress
import pandas as pd
import pyarrow as pa

# Check Dask version
print(f"Dask version: {dask.__version__}")

In [ ]:
# Initialize Dask client
# In ODP Workspaces, this connects to the pre-configured Dask scheduler
client = Client()

# Display cluster info
print(f"Dashboard: {client.dashboard_link}")
client

## 2. Connect to ODP Dataset

In [ ]:
from odp.client import Client as ODPClient

# Initialize ODP client
odp = ODPClient()

# Use a dataset with substantial data volume
# GLODAP - globally calibrated ocean carbon data (1M+ points)
DATASET_ID = "1d801817-742b-4867-82cf-5597673524eb"  # PGS Biota - adjust as needed

dataset = odp.dataset(DATASET_ID)

# Check dataset size
stats = dataset.table.stats()
if stats:
    print(f"Dataset rows: {stats.num_rows:,}")
    print(f"Dataset size: {stats.size / 1024 / 1024:.2f} MB")

## 3. Streaming ODP Data into Dask

The ODP SDK supports streaming via `.batches()` which returns PyArrow RecordBatches. We can convert these to a Dask DataFrame for distributed processing.

In [ ]:
def odp_to_dask(dataset, filter_expr=None, cols=None, partitions=4):
    """
    Stream ODP dataset into a Dask DataFrame.
    
    Args:
        dataset: ODP dataset object
        filter_expr: Optional filter expression
        cols: Optional list of columns to select
        partitions: Number of partitions to create
    
    Returns:
        Dask DataFrame
    """
    # Collect batches into pandas DataFrames
    dfs = []
    
    select_args = {}
    if filter_expr:
        select_args['filter'] = filter_expr
    if cols:
        select_args['cols'] = cols
    
    for batch in dataset.table.select(**select_args).batches():
        dfs.append(batch.to_pandas())
    
    if not dfs:
        return None
    
    # Concatenate and convert to Dask
    full_df = pd.concat(dfs, ignore_index=True)
    dask_df = dd.from_pandas(full_df, npartitions=partitions)
    
    return dask_df

# Load dataset into Dask DataFrame
print("Loading dataset into Dask...")
ddf = odp_to_dask(dataset, partitions=4)

if ddf is not None:
    print(f"Dask DataFrame created with {ddf.npartitions} partitions")
    print(f"Columns: {list(ddf.columns)}")
else:
    print("No data loaded - check dataset ID")

In [ ]:
# Preview the data (lazy - only computes first partition)
ddf.head()

## 4. Lazy Computation with Dask

Dask uses lazy evaluation - operations build a task graph that executes only when you call `.compute()`.

In [ ]:
# Define a computation (lazy - not executed yet)
# Example: Count observations by scientific name
species_counts = ddf.groupby('scientificName').size()

# This just shows the task graph structure
print("Task graph created (not yet computed)")
print(f"Type: {type(species_counts)}")

In [ ]:
# Execute the computation
result = species_counts.compute()

print(f"\nSpecies observation counts:")
print(result.sort_values(ascending=False).head(10))

## 5. Parallel Aggregations

Dask excels at parallel aggregations across partitions.

In [ ]:
# Multiple aggregations in parallel
# Adjust column names based on your dataset schema

# Check available numeric columns
numeric_cols = ddf.select_dtypes(include=['number']).columns.tolist()
print(f"Numeric columns: {numeric_cols}")

In [ ]:
# Example: Statistics by life stage (if column exists)
if 'lifeStage' in ddf.columns and 'minimumDepthInMeters' in ddf.columns:
    depth_stats = ddf.groupby('lifeStage').agg({
        'minimumDepthInMeters': ['mean', 'min', 'max', 'count']
    }).compute()
    
    print("Depth statistics by life stage:")
    print(depth_stats)
else:
    print("Columns 'lifeStage' or 'minimumDepthInMeters' not found.")
    print(f"Available columns: {list(ddf.columns)}")

## 6. Parallel Apply for Custom Functions

Use `map_partitions` to apply custom functions across partitions in parallel.

In [ ]:
def process_partition(df):
    """
    Custom processing function applied to each partition.
    Example: Extract year from eventDate and count observations.
    """
    result = df.copy()
    
    if 'eventDate' in df.columns:
        # Parse dates and extract year
        result['eventDate'] = pd.to_datetime(result['eventDate'], errors='coerce')
        result['year'] = result['eventDate'].dt.year
    
    return result

# Apply function across all partitions in parallel
processed_ddf = ddf.map_partitions(process_partition)

# Check result
if 'year' in processed_ddf.columns:
    yearly_counts = processed_ddf.groupby('year').size().compute()
    print("Observations by year:")
    print(yearly_counts.sort_index())

## 7. Memory-Efficient Processing Pattern

For very large datasets, process in chunks and aggregate results progressively.

In [ ]:
def process_large_dataset_streaming(dataset, chunk_processor, filter_expr=None):
    """
    Process large ODP dataset in streaming fashion with Dask.
    
    Args:
        dataset: ODP dataset
        chunk_processor: Function that takes a DataFrame and returns aggregated result
        filter_expr: Optional filter
    
    Returns:
        Combined results from all chunks
    """
    results = []
    
    select = dataset.table.select(filter_expr) if filter_expr else dataset.table.select()
    
    for i, batch in enumerate(select.batches()):
        df = batch.to_pandas()
        
        # Process chunk with Dask (useful for complex operations)
        ddf_chunk = dd.from_pandas(df, npartitions=2)
        chunk_result = chunk_processor(ddf_chunk)
        results.append(chunk_result)
        
        print(f"Processed batch {i+1}: {len(df)} rows")
    
    return results

# Example chunk processor
def count_by_species(ddf_chunk):
    if 'scientificName' in ddf_chunk.columns:
        return ddf_chunk.groupby('scientificName').size().compute()
    return pd.Series()

# Process dataset
print("Processing dataset in streaming mode...")
chunk_results = process_large_dataset_streaming(dataset, count_by_species)

# Combine results
if chunk_results:
    combined = pd.concat(chunk_results).groupby(level=0).sum()
    print(f"\nCombined species counts ({len(combined)} species):")
    print(combined.sort_values(ascending=False).head(10))

## 8. Geospatial Processing with Dask

Combine ODP's geospatial filtering with Dask's parallel processing.

In [ ]:
# Define regions of interest
regions = {
    "north_sea": "POLYGON((-5 51, 9 51, 9 62, -5 62, -5 51))",
    "norwegian_sea": "POLYGON((-5 62, 15 62, 15 72, -5 72, -5 62))",
    "barents_sea": "POLYGON((15 68, 40 68, 40 80, 15 80, 15 68))"
}

def process_region(dataset, region_name, wkt_polygon, geometry_col='footprintWKT'):
    """
    Process data for a specific geographic region.
    """
    try:
        # Use ODP's geospatial filter
        filter_expr = f"{geometry_col} within $area"
        
        dfs = []
        for batch in dataset.table.select(filter_expr, vars={"area": wkt_polygon}).batches():
            dfs.append(batch.to_pandas())
        
        if dfs:
            df = pd.concat(dfs, ignore_index=True)
            return {
                "region": region_name,
                "observation_count": len(df),
                "unique_species": df['scientificName'].nunique() if 'scientificName' in df.columns else 0
            }
    except Exception as e:
        print(f"Error processing {region_name}: {e}")
    
    return {"region": region_name, "observation_count": 0, "unique_species": 0}

# Process regions (could be parallelized with Dask delayed)
from dask import delayed

# Create delayed tasks for each region
delayed_results = [
    delayed(process_region)(dataset, name, wkt)
    for name, wkt in regions.items()
]

# Execute in parallel
print("Processing regions in parallel...")
region_stats = dask.compute(*delayed_results)

# Display results
region_df = pd.DataFrame(region_stats)
print("\nRegion Statistics:")
print(region_df)

## 9. Monitoring and Performance

Use the Dask dashboard to monitor task execution.

In [ ]:
# Display dashboard link
print(f"Dask Dashboard: {client.dashboard_link}")
print("\nOpen this URL to monitor:")
print("- Task stream: real-time task execution")
print("- Progress: overall computation progress")
print("- Workers: resource utilization per worker")
print("- Memory: memory usage across cluster")

In [ ]:
# Check cluster status
print("Cluster Info:")
print(f"  Workers: {len(client.scheduler_info()['workers'])}")
print(f"  Total threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")
print(f"  Total memory: {sum(w['memory_limit'] for w in client.scheduler_info()['workers'].values()) / 1e9:.1f} GB")

## 10. Cleanup

In [ ]:
# Close Dask client when done
client.close()
print("Dask client closed.")

## Summary

This notebook demonstrated:

1. **Dask Client Setup** - Connecting to the ODP Workspace Dask cluster
2. **ODP to Dask** - Converting streaming ODP data to Dask DataFrames
3. **Lazy Evaluation** - Building computation graphs before execution
4. **Parallel Aggregations** - Group-by operations across partitions
5. **Custom Processing** - Using `map_partitions` for parallel apply
6. **Streaming Pattern** - Memory-efficient processing for large datasets
7. **Geospatial + Parallel** - Combining ODP spatial filters with Dask parallelism
8. **Monitoring** - Using the Dask dashboard for performance visibility

## Next Steps

- **02_geospatial_analysis.ipynb**: H3 hexagonal aggregation and mapping
- **03_data_pipeline.ipynb**: File ingest workflows
- **04_multi_dataset_join.ipynb**: Cross-dataset analysis

## Resources

- [Dask Documentation](https://docs.dask.org/)
- [Dask DataFrame API](https://docs.dask.org/en/stable/dataframe.html)
- [ODP Python SDK](https://docs.hubocean.earth/python_sdk/intro/)
- [Dask Best Practices](https://docs.dask.org/en/stable/best-practices.html)